# Results of the closure strategy simulations

This notebook contains the results obtained from the simulation of a closure strategy. The results are compared with the baseline results.

(The first time you should execute all the cell, then if you want just analyze results can skip cell group 1)

In [ ]:
import json
from result_utils import *
import sumolib
import pandas as pd
import html
import osmnx as ox
import networkx as nx
import matplotlib as mpl
import folium
import numpy as np
from ast import literal_eval
import glob
from shapely.geometry import LineString

#### parameters

In [ ]:
k = '100'
exp_name = 'MIX_top'+k
exp_rand = 'rand'+k
city = 'Milano_big'

# road network path
road_network_path = "../data/road_net/"+city+"/"+city+"_road_network.net.xml"

# removed edges csv
removed_road_path = '../data/simulations/'+city+'/'+exp_name+'/'+city+'_road_'+exp_name+'.csv'
removed_rand_road_path = '../data/simulations/'+city+'/'+exp_name+'/rand/*.csv'

path_road_edge_mapping = '../data/road_net/'+city+'/'+city+'_road_edge_map.csv'

# routed paths folder baseline
rou_path_folder = '../data/simulations/'+city+'/baseline/routed_paths/'

# Experiment folders
folder_experiments = "../data/simulations/"+city+"/"+exp_name+"/sumo_out/"
folder_experiments_rand = "../data/simulations/"+city+"/"+exp_name+"/rand/sumo_out/"
folder_baseline = '../data/simulations/'+city+'/baseline/sumo_out/'

# output paths
path_results = "../data/simulations/"+city+"/"+exp_name+"/results/"
path_plots = "../data/simulations/"+city+"/"+exp_name+"/plots/"

Create a dictionary that associates configuration and filenames

In [ ]:
rand_removed_road_edge_list = []
for file in glob.glob(removed_rand_road_path):
    rand_removed_road_edge = pd.read_csv(file)
    rand_removed_road_edge_list.append(rand_removed_road_edge)

In [ ]:
len(rand_removed_road_edge_list)

In [ ]:
exp_rand_list = []
for i in range(len(rand_removed_road_edge_list)):
    exp_rand_list.append(exp_rand+'_'+str(i))

In [ ]:
dict_exps = create_dict_exps(folder_experiments, exp_name)
dict_baseline = create_dict_exps(folder_baseline, 'baseline')
dict_rand = {}
for exp in exp_rand_list:
    dict_rand_i = create_dict_exps_rand(folder_experiments_rand, exp)
    dict_rand.update(dict_rand_i)

## 1. Compute road aggregation for each experiment

Aggregate the results of the experiments to road level.

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)

In [ ]:
def create_road_measures(dict_exps, folder_experiments, main_experiment_name, road_edge_map):
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/edge_measures.csv")
        merged = pd.merge(road_edge_map, exp_df, on=['edge_id'])
        grouped = merged.groupby('road').agg({'edge_id': lambda x: x.tolist(),
                                              'edge_len': 'sum',
                                              'total_co2': 'sum',
                                              'total_nox': 'sum',
                                              'total_fuel': 'sum',
                                              'total_v_edge': 'sum'})
        grouped.reset_index(inplace=True)
        grouped.to_csv(folder_experiments+exp_folder_name+"/road_measures.csv", index=False)

In [ ]:
create_road_measures(dict_exps, folder_experiments, exp_name, road_edge_map)
for exp in exp_rand_list:
    create_road_measures(dict_rand, folder_experiments_rand, exp, road_edge_map)

## 2. Results by roads

Visualization of the results by road.

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)

In [ ]:
removed_roads = pd.read_csv(removed_road_path)

In [ ]:
removed_edges = np.array([])

for edge_list in removed_roads['edge_id']:
    removed_edges = np.append(removed_edges, literal_eval(edge_list))

In [ ]:
#removed_roads_rand = pd.read_csv(removed_road_rand_path)
removed_edges_rand_list = []
for removed_roads_rand in rand_removed_road_edge_list:
    removed_edges_rand = np.array([])

    for edge_list in removed_roads_rand['edge_id']:
        removed_edges_rand = np.append(removed_edges_rand, literal_eval(edge_list))
    removed_edges_rand_list.append(removed_edges_rand)

In [ ]:
# Aggregate all experiments results and compute the mean and the std of the 'total_of' column.
# It returns a dictionary with keys = roadnames and list [mean, std].

def create_dict_total_per_road(dict_exps, folder_experiments, main_experiment_name, total_of):
    dict_total = {}
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/road_measures.csv")
        
        for ind, row in exp_df.iterrows():
            if row['road'] in dict_total:
                dict_total[row['road']].append(row[total_of])
            else:
                dict_total[row['road']] = [row[total_of]]
    
    list_df = []
    for road, total in dict_total.items():
        list_df.append([road, np.array(total).mean(), np.array(total).std()])
    df = pd.DataFrame(list_df, columns=['road', 'mean', 'std'])
        
    return df

In [ ]:
# Aggregate all experiments results and compute the mean and the std of the 'total_of' column.
# It returns a dictionary with keys = roadnames and list [mean, std].

def create_dict_total_per_road_rand(dict_exps, folder_experiments, total_of):
    dict_total = {}
    for exp_id, exp_dict in dict_exps.items():
        for n_exp, exp_name in exp_dict.items():
        
            exp_df = pd.read_csv(folder_experiments+exp_name+"/road_measures.csv")

            for ind, row in exp_df.iterrows():
                if row['road'] in dict_total:
                    dict_total[row['road']].append(row[total_of])
                else:
                    dict_total[row['road']] = [row[total_of]]
    
    list_df = []
    for road, total in dict_total.items():
        list_df.append([road, np.array(total).mean(), np.array(total).std()])
    df = pd.DataFrame(list_df, columns=['road', 'mean', 'std'])
        
    return df

In [ ]:
# count edges used in each experiment

def count_used_edges(dict_exps, folder_experiments, main_experiment_name):
    exp_edge_map = {}
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/edge_measures.csv")
        
        used_edges = exp_df[(exp_df['total_co2'] != 0) & (~exp_df['edge_id'].str.startswith(':'))]['edge_id'].count()
        
        if main_experiment_name in exp_edge_map:
            exp_edge_map[main_experiment_name].append(used_edges)
        else:
            exp_edge_map[main_experiment_name] = [used_edges]
    
    list_df = []
    for exp, edges in exp_edge_map.items():
        list_df.append([exp, np.array(edges).mean(), np.array(edges).std()])
    df = pd.DataFrame(list_df, columns=['exp', 'mean', 'std'])
        
    return df

In [ ]:
# count used edges in rand experiments

def count_used_edges_rand(dict_exps, folder_experiments, main_experiment_name):
    exp_edge_map = {}
    for exp_id, exp_dict in dict_exps.items():
        for n_exp, exp_name in exp_dict.items():
        
            exp_df = pd.read_csv(folder_experiments+exp_name+"/edge_measures.csv")

            used_edges = exp_df[(exp_df['total_co2'] != 0) & (~exp_df['edge_id'].str.startswith(':'))]['edge_id'].count()
        
            if main_experiment_name in exp_edge_map:
                exp_edge_map[main_experiment_name].append(used_edges)
            else:
                exp_edge_map[main_experiment_name] = [used_edges]
    
    list_df = []
    for exp, edges in exp_edge_map.items():
        list_df.append([exp, np.array(edges).mean(), np.array(edges).std()])
    df = pd.DataFrame(list_df, columns=['exp', 'mean', 'std'])
    
    return df

In [ ]:
# count the number of vehicles that passed through a removed edge

def count_impacted_vehicles(rou_path_folder, removed_edges):
    imp_vehicle = []

    for rou_file in os.listdir(rou_path_folder):
        rp_tmp = list(sumolib.xml.parse(rou_path_folder+rou_file, 'route'))

        v_counter = 0
        rem_edges_set = set(removed_edges)
        for p in rp_tmp:
            edgs = set(p.edges.split(' '))
            if len(edgs.intersection(rem_edges_set)) != 0:
                v_counter += 1
        
        imp_vehicle.append(v_counter)

    return [np.mean(imp_vehicle), np.std(imp_vehicle)]


In [ ]:
# count the number of vehicles that passed through a removed edge for random experiment

def count_impacted_vehicles_rand(rou_path_folder, removed_edges_list):
    imp_vehicle = []

    for rou_file in os.listdir(rou_path_folder):
        rp_tmp = list(sumolib.xml.parse(rou_path_folder+rou_file, 'route'))

        for i in range(len(removed_edges_list)):
            v_counter = 0
            rem_edges_set = set(removed_edges_list[i])
            for p in rp_tmp:
                edgs = set(p.edges.split(' '))
                if len(edgs.intersection(rem_edges_set)) != 0:
                    v_counter += 1
            imp_vehicle.append(v_counter)
        
    return [np.mean(imp_vehicle), np.std(imp_vehicle)]


### 1. Total CO2 per road

Statistics of the total CO2 per road.

In [ ]:
used_edges_baseline = count_used_edges(dict_baseline, folder_baseline, 'baseline')
used_edges = count_used_edges(dict_exps, folder_experiments, exp_name)
used_edges_rand = count_used_edges_rand(dict_rand, folder_experiments_rand, exp_rand)

In [ ]:
imp_veichles = count_impacted_vehicles(rou_path_folder, removed_edges)
imp_veichles_rand = count_impacted_vehicles_rand(rou_path_folder, removed_edges_rand_list)

In [ ]:
df_co2 = create_dict_total_per_road(dict_exps, folder_experiments, exp_name, 'total_co2')
df_co2_baseline = create_dict_total_per_road(dict_baseline, folder_baseline, 'baseline', 'total_co2')
df_co2_rand = create_dict_total_per_road_rand(dict_rand, folder_experiments_rand, 'total_co2')

In [ ]:
print('Total CO2 {}: {:.3e} \u00B1 {:.3}'.format(exp_name, df_co2['mean'].sum(), np.sqrt(df_co2['std'].pow(2).sum())))
print('Total CO2 {}: {:.3e} \u00B1 {:.3}'.format(exp_rand, df_co2_rand['mean'].sum(), np.sqrt(df_co2_rand['std'].pow(2).sum())))
print('Total CO2 baseline: {:.3e} \u00B1 {:.3}'.format(df_co2_baseline['mean'].sum(), np.sqrt(df_co2_baseline['std'].pow(2).sum())))

In [ ]:
print('Total CO2 {} wrt baseline: {:.2f}%'.format(exp_name, df_co2['mean'].sum()/df_co2_baseline['mean'].sum()*100-100))
print('Total CO2 {} wrt baseline: {:.2f}%'.format(exp_rand, df_co2_rand['mean'].sum()/df_co2_baseline['mean'].sum()*100-100))

In [ ]:
# save stats
df_stats = pd.DataFrame({'experiment': ['baseline', exp_name, exp_rand],
                         'co2_mean': [df_co2_baseline['mean'].sum(), df_co2['mean'].sum(), df_co2_rand['mean'].sum()],
                         'co2_std': [np.sqrt(df_co2_baseline['std'].pow(2).sum()), np.sqrt(df_co2['std'].pow(2).sum()), np.sqrt(df_co2_rand['std'].pow(2).sum())],
                         '%_wrt_baseline': [0, df_co2['mean'].sum()/df_co2_baseline['mean'].sum()*100-100, df_co2_rand['mean'].sum()/df_co2_baseline['mean'].sum()*100-100],
                         'edges_mean': [used_edges_baseline['mean'].item(), used_edges['mean'].item(), used_edges_rand['mean'].item()],
                         'edges_std': [used_edges_baseline['std'].item(), used_edges['std'].item(), used_edges_rand['std'].item()],
                         'impacted_v_mean': [0, imp_veichles[0], imp_veichles_rand[0]],
                         'impacted_v_std': [0, imp_veichles[1], imp_veichles_rand[1]],
                         })
df_stats.to_csv(path_results+exp_name+'_total_co2.csv', index=False)

Compute CO2 per meter of each road

In [ ]:
road_len = road_edge_map.groupby(by=['road']).sum(numeric_only=True).reset_index()

In [ ]:
df_co2 = pd.merge(df_co2, road_len, on=['road'])
df_co2_rand = pd.merge(df_co2_rand, road_len, on=['road'])
df_co2_baseline = pd.merge(df_co2_baseline, road_len, on=['road'])

In [ ]:
df_co2['mean_len'] = df_co2['mean']/df_co2['edge_len']
df_co2['std_len'] = df_co2['std']/df_co2['edge_len']
df_co2_rand['mean_len'] = df_co2_rand['mean']/df_co2_rand['edge_len']
df_co2_rand['std_len'] = df_co2_rand['std']/df_co2_rand['edge_len']
df_co2_baseline['mean_len'] = df_co2_baseline['mean']/df_co2_baseline['edge_len']
df_co2_baseline['std_len'] = df_co2_baseline['std']/df_co2_baseline['edge_len']

In [ ]:
df_co2.sort_values(by=['mean_len', 'std_len'], ascending=False, inplace=True)

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(range(df_co2.shape[0]), df_co2['mean_len'], yerr=df_co2['std_len'])
plt.title('CO2\m per road '+exp_name)
plt.xlabel('road_id')
plt.ylabel('CO2 per meter (mg\m)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(df_co2['road'].iloc[:50].apply(html.unescape), df_co2['mean_len'].iloc[:50], yerr=df_co2['std_len'].iloc[:50])
plt.title('CO2\m per road '+exp_name)
plt.xlabel('roadname')
plt.ylabel('CO2 per meter (mg\m)')
plt.xticks(rotation='vertical')
plt.savefig(path_plots+'CO2_'+exp_name+'_first50.png', bbox_inches ="tight")
plt.show()

In [ ]:
df_co2_vs = pd.merge(df_co2, df_co2_baseline, on=['road'], suffixes=('_'+exp_name, '_'))
df_co2_vs = pd.merge(df_co2_rand, df_co2, on=['road'], suffixes=('_'+exp_rand, '_'+exp_name))
df_co2_vs = pd.merge(df_co2_vs, df_co2_baseline, on=['road'])

In [ ]:
df_co2_vs.sort_values(by=['mean_len_'+exp_name, 'std_len_'+exp_name], ascending=False, inplace=True)

In [ ]:
df_co2_vs2 = df_co2_vs
df_co2_vs2['road'] = df_co2_vs2['road'].apply(html.unescape)
df_co2_vs2.iloc[:50].plot(kind='bar', x='road', y=['mean_len_'+exp_name, 'mean_len_'+exp_rand, 'mean_len'],
                                 title='CO2\m '+exp_name+' vs CO2\m '+exp_rand+' CO\m2 baseline', ylabel='CO2 per meter (mg\m)', 
                                 width=0.8, rot=90, figsize=(10,3))
plt.savefig(path_plots+'CO2_comparison.png', bbox_inches ="tight")
plt.show()

### 2. Bland Altman plot

Comparison of emission per road between baseline and the closure strategy.

In [ ]:
df_co2_vs['delta'] = df_co2_vs['mean_len_'+exp_name] - df_co2_vs['mean_len']
df_co2_vs['delta_rand'] = df_co2_vs['mean_len_'+exp_rand] - df_co2_vs['mean_len']

In [ ]:
# compute the colormap
norm = mpl.colors.TwoSlopeNorm(vmin=df_co2_vs['delta'].min(), vmax=df_co2_vs['delta'].max(), vcenter=0)
cmap = plt.cm.get_cmap('seismic')
colors = cmap(norm(df_co2_vs['delta']))

In [ ]:
df_co2_vs = df_co2_vs.sample(frac=1)

In [ ]:
plt.scatter(range(df_co2_vs.shape[0]), df_co2_vs['delta'], s=5, color=cmap(norm(df_co2_vs['delta'])))
plt.axhline(0, color='black', linestyle='--')
plt.title('CO2\m '+exp_name+' - CO2\m baseline')
plt.ylabel('\u0394 CO2 per meter (mg\m)')
plt.xlabel('Road_id')
plt.savefig(path_plots+'Bland_Altman_CO2_'+exp_name+'.png', bbox_inches ="tight")
plt.show()

In [ ]:
# compute the colormap
norm = mpl.colors.TwoSlopeNorm(vmin=df_co2_vs['delta_rand'].min(), vmax=df_co2_vs['delta_rand'].max(), vcenter=0)
cmap = plt.cm.get_cmap('seismic')
colors = cmap(norm(df_co2_vs['delta_rand']))

In [ ]:
plt.scatter(range(df_co2_vs.shape[0]), df_co2_vs['delta_rand'], s=5, color=cmap(norm(df_co2_vs['delta_rand'])))
plt.axhline(0, color='black', linestyle='--')
plt.title('CO2\m '+exp_rand+' - CO2\m baseline')
plt.ylabel('\u0394 CO2 per meter (mg\m)')
plt.xlabel('Road_id')
plt.savefig(path_plots+'Bland_Altman_CO2_'+exp_rand+'.png', bbox_inches ="tight")
plt.show()

### 3. Emission per road type

Visualization of the level of CO2 emissions grouped by road type.

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

In [ ]:
# find type for each edge

edge_id = []
types = []

for edge in road_network.getEdges():
    edge_id.append(edge.getID())
    types.append(edge.getType())

In [ ]:
# df edge-type

df_edge_type = pd.DataFrame({'edge_id': edge_id, 'type': types})
df_edge_type = pd.merge(df_edge_type, road_edge_map, on=['edge_id'])

In [ ]:
# assign to each road the most representative type by length

df_group = df_edge_type.groupby(by=['road', 'type']).agg(count=('edge_len', 'count'),
                                                         edge_len=('edge_len', 'sum')).reset_index()
idx_max = df_group.groupby(by='road')['edge_len'].idxmax()
road_type_map = df_group.loc[idx_max][['road', 'type']]

In [ ]:
# merge with results dataframe and cleaning

df_co2_type = pd.merge(df_co2_vs, road_type_map, on=['road'], how='left')
df_co2_type['type'].fillna('highway.unclassified', inplace=True)


df_type_plot = df_co2_type.groupby(by=['type']).agg(road_len=('edge_len', 'sum'),
                                                    mean=('mean', 'sum'),
                                                    mean_top=('mean_'+exp_name, 'sum'),
                                                    mean_rand=('mean_'+exp_rand, 'sum')).reset_index()
df_type_plot['type'] = df_type_plot['type'].apply(lambda s: s.replace('highway.', ''))
df_type_plot.sort_values(by=['road_len'], ascending=False, inplace=True)
df_type_plot.reset_index(inplace=True)
df_type_plot.drop('index', axis=1, inplace=True)

In [ ]:
df_type_plot.head()

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 3))
ax2 = ax1.twinx()

# plot
df_type_plot['road_len'].plot(kind='bar', color='lightslategrey', ax=ax1)
df_type_plot['mean'].plot(kind='line', color='tab:blue', marker='o', markersize=3, ax=ax2)
df_type_plot['mean_top'].plot(kind='line', color='orange', marker='o', markersize=3, ax=ax2)
df_type_plot['mean_rand'].plot(kind='line', color='firebrick', marker='o', markersize=3, ax=ax2)

# bar ax
ax1.bar_label(ax1.containers[0], fontsize=7)
ax1.set_xticklabels(df_type_plot['type'], rotation=45)
ax1.set_ylabel('Length of type (m)')

# line ax
ax2.legend(['baseline', exp_name, exp_rand])
ax2.set_ylabel('CO2 emission (mg)')
ax2.set_title('Emission per road type ('+exp_name+')')

plt.savefig(path_plots+'CO2_road_type_'+exp_name+'.png', bbox_inches ="tight")
plt.show()

### 4. Gini index CO2 per road

In [ ]:
def gini(array):
    """Gini coefficient for array >= 0. Faster than the standard Gini"""
    array.sort()
    # values can't be zero
    array = array + 0.0000001
    index = np.array(np.arange(1, array.shape[0]+1))
    n = array.shape[0]
    return (np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))

In [ ]:
def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [ ]:
def create_dict_gini(dict_exps, folder_experiments, main_experiment_name, gini_of):
    dict_gini = {}
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/road_measures.csv")
        gini_array = np.array(exp_df[gini_of])
        dict_gini[exp_id] = gini(gini_array)
    return dict_gini

In [ ]:
def create_dict_gini_rand(dict_exps, folder_experiments, gini_of):
    dict_gini = {}
    for exp_id, exp_dict in dict_exps.items():
        for n_exp, exp_name in exp_dict.items():
            exp_df = pd.read_csv(folder_experiments+exp_name+"/road_measures.csv")
            gini_array = np.array(exp_df[gini_of])
            dict_gini[exp_id+'_'+n_exp] = gini(gini_array)
    return dict_gini

In [ ]:
dict_gini_co2 = create_dict_gini(dict_exps, folder_experiments, exp_name, 'total_co2')
dict_gini_co2_baseline = create_dict_gini(dict_baseline, folder_baseline, 'baseline', 'total_co2')
dict_gini_co2_rand = create_dict_gini_rand(dict_rand, folder_experiments_rand, 'total_co2')

with open(path_results+'co2_gini.json', 'w') as fp:
    json.dump(dict_gini_co2, fp)
with open(path_results+'co2_gini_rand.json', 'w') as fp:
    json.dump(dict_gini_co2_rand, fp)

In [ ]:
gini_co2 = np.array(list(dict_gini_co2.values()))
gini_baseline = np.array(list(dict_gini_co2_baseline.values()))
gini_co2_rand = np.array(list(dict_gini_co2_rand.values()))

print('Gini coefficient CO2 {} mean: {:.5} \u00B1 {:.3}'.format(exp_name, gini_co2.mean(), gini_co2.std()))
print('Gini coefficient CO2 {} mean: {:.5} \u00B1 {:.3}'.format(exp_rand, gini_co2_rand.mean(), gini_co2_rand.std()))
print('Gini coefficient CO2 baseline mean: {:.5} \u00B1 {:.3}'.format(gini_baseline.mean(), gini_baseline.std()))

In [ ]:
# save stats

df_gini = pd.DataFrame({'experiment': ['baseline', exp_name, exp_rand],
                        'gini_co2_mean': [gini_baseline.mean(), gini_co2.mean(), gini_co2_rand.mean()],
                        'gini_co2_std': [gini_baseline.std(), gini_co2.std(), gini_co2_rand.std()]})
df_stats.to_csv(path_results+exp_name+'_gini_co2.csv', index=False)

### 5. CO2 plot OSMnx

Visualization on map of the results obtained by the closure strategy, compared with the baseline

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

In [ ]:
len(road_network.getEdges())

In [ ]:
G = nx.MultiDiGraph()

for edge in road_network.getEdges():
    node_from = edge.getFromNode().getID()
    node_to = edge.getToNode().getID()
    geom = [list(x) for x in edge.getShape()]
    G.add_edge(node_from, node_to, key=edge.getID(), length=edge.getLength(), geometry=LineString(geom))
    
G.graph.update({'crs': 'epsg:3857'})

In [ ]:
#ox.plot.plot_graph(G, node_size=0, bgcolor='white', edge_color='lightgrey', edge_linewidth=0.5, figsize=(8,8), dpi=500)

In [ ]:
def plot_df(df_exps, road_edge_map, road_net):
    road_edge_map_no_intern = road_edge_map[~road_edge_map['edge_id'].astype(str).str.startswith(':')]
    road_edge_mean_map = pd.merge(road_edge_map_no_intern, df_exps, on=['road'])
    
    # create column with tuple of edges of the graph (u,v,key)
    edge_graph_list = []
    for edge in road_edge_mean_map['edge_id']:
        from_node = road_net.getEdge(edge).getFromNode().getID()
        to_node = road_net.getEdge(edge).getToNode().getID()
        edge_graph_list.append((from_node, to_node, edge))
        
    road_edge_mean_map['edge_graph'] = edge_graph_list
    
    return road_edge_mean_map

In [ ]:
df_co2_plot = plot_df(df_co2_vs, road_edge_map, road_network)

In [ ]:
# exclude removed roads from divergent colormap
df_co2_plot.loc[df_co2_plot['edge_id'].isin(removed_edges), 'delta'] = 0

In [ ]:
def add_attribute_to_graph(graph, df_plot, attr_name, plot_col):
    # edge[0] = node_from, edge[1] = node_to, edge[2] = key = edge_id 
    
    # Initialize co2 attribute in the graph
    for edge in graph.edges:
        G[edge[0]][edge[1]][edge[2]][attr_name] = None
        
    # Set co2 attribute based on some value per road
    for edge, value in zip(df_plot['edge_graph'], df_plot[plot_col]):
        graph[edge[0]][edge[1]][edge[2]][attr_name] = value

In [ ]:
add_attribute_to_graph(G, df_co2_plot, 'co2_'+exp_name, 'mean_'+exp_name)
add_attribute_to_graph(G, df_co2_plot, 'co2_'+exp_rand, 'mean_'+exp_rand)
add_attribute_to_graph(G, df_co2_plot, 'co2_len_'+exp_name, 'mean_len_'+exp_name)
add_attribute_to_graph(G, df_co2_plot, 'co2_len_'+exp_rand, 'mean_len_'+exp_rand)
add_attribute_to_graph(G, df_co2_plot, 'delta_co2_'+exp_name, 'delta')
add_attribute_to_graph(G, df_co2_plot, 'delta_co2_'+exp_rand, 'delta_rand')

In [ ]:
# colormap osmnx
ec_co2 = ox.plot.get_edge_colors_by_attr(G, attr='co2_'+exp_name, cmap='inferno_r', na_color='white')
ec_co2_rand = ox.plot.get_edge_colors_by_attr(G, attr='co2_'+exp_rand, cmap='inferno_r', na_color='white')
ec_co2_len = ox.plot.get_edge_colors_by_attr(G, attr='co2_len_'+exp_name, cmap='inferno_r', na_color='white')
ec_co2_len_rand = ox.plot.get_edge_colors_by_attr(G, attr='co2_len_'+exp_rand, cmap='inferno_r', na_color='white')
ec_delta = ox.plot.get_edge_colors_by_attr(G, attr='delta_co2_'+exp_name, cmap='RdYlGn_r', na_color='lightyellow')
ec_delta_rand = ox.plot.get_edge_colors_by_attr(G, attr='delta_co2_'+exp_rand, cmap='RdYlGn_r', na_color='lightyellow')

In [ ]:
# Remap osmnx colormap to get divergent colormap centered in 0
norm = mpl.colors.TwoSlopeNorm(vmin=df_co2_plot['delta'].min(),
                               vmax=df_co2_plot['delta'].max(),
                               vcenter=0)
cmap = plt.cm.get_cmap('RdYlGn_r')

for k,v in ec_delta.items():
    if v != 'lightyellow':
        ec_delta[k] = tuple(cmap(norm(df_co2_plot[df_co2_plot['edge_graph']==k]['delta']))[0])

In [ ]:
# Remap osmnx colormap to get divergent colormap centered in 0
norm = mpl.colors.TwoSlopeNorm(vmin=df_co2_plot['delta_rand'].min(),
                               vmax=df_co2_plot['delta_rand'].max(),
                               vcenter=0)
cmap = plt.cm.get_cmap('RdYlGn_r')

for k,v in ec_delta_rand.items():
    if v != 'lightyellow':
        ec_delta_rand[k] = tuple(cmap(norm(df_co2_plot[df_co2_plot['edge_graph']==k]['delta_rand']))[0])

In [ ]:
# Set color of removed edges
for edge in removed_edges:
    if not edge.startswith(':'):
        edge_osm = df_co2_plot[df_co2_plot['edge_id']==edge]['edge_graph']
        ec_co2[edge_osm] = 'royalblue'
        ec_co2_len[edge_osm] = 'royalblue'
        ec_delta[edge_osm] = 'black'

In [ ]:
# Set color of removed edges
#for edge in removed_edges_rand:
#    if not edge.startswith(':'):
#        edge_osm = df_co2_plot[df_co2_plot['edge_id']==edge]['edge_osmnx']
#        ec_co2_rand[edge_osm] = 'black'
#        ec_co2_len_rand[edge_osm] = 'black'
#        ec_delta_rand[edge_osm] = 'black'

Overall CO2 emissions for top and random experiment

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_co2, show=False)

# colorbar
norm = mpl.colors.Normalize(vmin=min(v for v in nx.get_edge_attributes(G, 'co2_'+exp_name).values() if v is not None),
                            vmax=max(v for v in nx.get_edge_attributes(G, 'co2_'+exp_name).values() if v is not None))
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='inferno_r'), ax=ax, shrink=0.5)
cbar.set_label('CO2 (mg)')

plt.title('CO2 emission per road '+exp_name)
plt.savefig(path_plots+'OSMnx_CO2_'+exp_name+'.png', bbox_inches ="tight")
plt.show()

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_co2_rand, show=False)

# colorbar
norm = mpl.colors.Normalize(vmin=min(v for v in nx.get_edge_attributes(G, 'co2_'+exp_rand).values() if v is not None),
                            vmax=max(v for v in nx.get_edge_attributes(G, 'co2_'+exp_rand).values() if v is not None))
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='inferno_r'), ax=ax, shrink=0.5)
cbar.set_label('CO2 (mg)')

plt.title('CO2 emission per road '+exp_rand)
plt.savefig(path_plots+'OSMnx_CO2_'+exp_rand+'.png', bbox_inches ="tight")
plt.show()

CO2 per meter for top and random experiment

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_co2_len, show=False)

# colorbar
norm = mpl.colors.Normalize(vmin=min(v for v in nx.get_edge_attributes(G, 'co2_len_'+exp_name).values() if v is not None),
                            vmax=max(v for v in nx.get_edge_attributes(G, 'co2_len_'+exp_name).values() if v is not None))
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='inferno_r'), ax=ax, shrink=0.5)
cbar.set_label('CO2 per meter (mg\m)')

plt.title('CO2\m emission per road '+exp_name)
plt.savefig(path_plots+'OSMnx_CO2_len_'+exp_name+'.png', bbox_inches ="tight")
plt.show()

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_co2_len_rand, show=False)

# colorbar
norm = mpl.colors.Normalize(vmin=min(v for v in nx.get_edge_attributes(G, 'co2_len_'+exp_rand).values() if v is not None),
                            vmax=max(v for v in nx.get_edge_attributes(G, 'co2_len_'+exp_rand).values() if v is not None))
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='inferno_r'), ax=ax, shrink=0.5)
cbar.set_label('CO2 per meter (mg\m)')

plt.title('CO2\m emission per road '+exp_rand)
plt.savefig(path_plots+'OSMnx_CO2_len_'+exp_rand+'.png', bbox_inches ="tight")
plt.show()

CO2 per meter emission wrt the baseline for top and random experiment

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_delta, show=False)

# colorbar
norm = mpl.colors.TwoSlopeNorm(vmin=min(v for v in nx.get_edge_attributes(G, 'delta_co2_'+exp_name).values() if v is not None),
                               vmax=max(v for v in nx.get_edge_attributes(G, 'delta_co2_'+exp_name).values() if v is not None),
                               vcenter=0)
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='RdYlGn_r'), ax=ax, shrink=0.5)
cbar.set_label('\u0394 CO2 per meter (mg\m)')

plt.title('CO2\m emission per road '+exp_name+' - CO2\m emission baseline')
plt.savefig(path_plots+'OSMnx_delta_CO2_'+exp_name+'.png', bbox_inches ="tight")
plt.show()

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec_delta_rand, show=False)

# colorbar
norm = mpl.colors.TwoSlopeNorm(vmin=min(v for v in nx.get_edge_attributes(G, 'delta_co2_'+exp_rand).values() if v is not None),
                                vmax=max(v for v in nx.get_edge_attributes(G, 'delta_co2_'+exp_rand).values() if v is not None),
                                vcenter=0)
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='RdYlGn_r'), ax=ax, shrink=0.5)
cbar.set_label('\u0394 CO2 per meter (mg\m)')

plt.title('CO2\m emission per road '+exp_rand+' - CO2\m emission baseline')
plt.savefig(path_plots+'OSMnx_delta_CO2_'+exp_rand+'.png', bbox_inches ="tight")
plt.show()

### 6. CO2 plot folium

In [ ]:
# generate a dataframe to plot a sumo network in folium
# df_exp: dataframe with roadname, mean, std
# road_edge_map: dataframe with map roadname - sumo edge_id
# value_of_cmap: column values to use for the colormap
# cmap_name: name of the colormap

def plot_df_folium(df_exp, road_net, road_edge_map, value_of_cmap, cmap_name, divergent=False):
    # merge roadname - edge_id with values to plot
    road_edge_map_no_intern = road_edge_map[~road_edge_map['edge_id'].astype(str).str.startswith(':')]
    df = pd.merge(road_edge_map_no_intern, df_exp, on=['road'])
    
    # compute the colormap
    if divergent:
        norm = mpl.colors.TwoSlopeNorm(vmin=df[value_of_cmap].min(),
                                    vmax=df[value_of_cmap].max(),
                                    vcenter=0)
    else:
        norm = mpl.colors.Normalize(vmin=df[value_of_cmap].min(),
                                    vmax=df[value_of_cmap].max())
    cmap = plt.cm.get_cmap(cmap_name)
    colors = cmap(norm(df[value_of_cmap]))
    colors = [mpl.colors.to_hex(c, keep_alpha=True) for c in colors]
    
    # compute the edge lonlat to be plotted in folium
    list_from = []
    list_to = []

    for edge in df['edge_id']:
        # Compute lat and lon, from and to, for each edge
        coord_node_from = road_net.getEdge(edge).getFromNode().getCoord()
        coord_node_to = road_net.getEdge(edge).getToNode().getCoord()
        lon_from, lat_from = road_net.convertXY2LonLat(coord_node_from[0], coord_node_from[1])
        lon_to, lat_to = road_net.convertXY2LonLat(coord_node_to[0], coord_node_to[1])

        list_from.append((lat_from, lon_from))
        list_to.append((lat_to, lon_to))
        
    # update the dataframe
    df['from'] = list_from
    df['to'] = list_to
    df['color'] = colors
    
    return df

In [ ]:
#df_plot_co2 = plot_df_folium(df_co2, road_network, road_edge_map, 'mean_len', 'inferno_r')
#df_plot_co2_rand = plot_df_folium(df_co2_rand, road_network, road_edge_map, 'mean_len', 'inferno_r')
#df_plot_delta_co2 = plot_df_folium(df_co2_vs, road_network, road_edge_map, 'delta', 'RdYlGn_r', divergent=True)
#df_plot_delta_co2_rand = plot_df_folium(df_co2_vs, road_network, road_edge_map, 'delta_rand', 'RdYlGn_r', divergent=True)

#### Experiment

In [ ]:
#m = folium.Map(location=[45.469262, 9.182007],
#               tiles='CartoDB Positron',
#               zoom_start=12,
#               attr='CartoDB')
               #png_enabled=True)

In [ ]:
#for index, row in df_plot_co2.iterrows():
#    if row['edge_id'] in removed_edges:
#        folium.PolyLine([row['from'], row['to']],
#                         tooltip=row['road']+' - CO2_m: {:.3e}'.format(row['mean_len']),
#                         color='black', weight=1, dashArray='5').add_to(m)
#    else:
#        folium.PolyLine([row['from'], row['to']],
#                         tooltip=row['road']+' - CO2_m: {:.3e}'.format(row['mean_len']),
#                         color=row['color'], weight=1.5).add_to(m)

In [ ]:
#m

In [ ]:
# Export map as html
#m.save(path_plots+'folium_map_CO2_'+exp_name+'.html')

In [ ]:
# Export map as png

#import io
#from PIL import Image

#img_data = m._to_png(5)
#img = Image.open(io.BytesIO(img_data))
#img.save(path_plots+'folium_map_CO2.png')

#### Random experiment

In [ ]:
#m = folium.Map(location=[45.469262, 9.182007],
#               tiles='CartoDB Positron',
#               zoom_start=12,
#               attr='CartoDB')
               #png_enabled=True)

In [ ]:
#for index, row in df_plot_co2_rand.iterrows():
    #if row['edge_id'] in removed_edges_rand:
    #    folium.PolyLine([row['from'], row['to']],
    #                     tooltip=row['road']+' - CO2_m: {:.3e}'.format(row['mean_len']),
    #                     color='black', weight=1, dashArray='5').add_to(m)
    #else:
#    folium.PolyLine([row['from'], row['to']],
#                     tooltip=row['road']+' - CO2_m: {:.3e}'.format(row['mean_len']),
#                     color=row['color'], weight=1.5).add_to(m)

In [ ]:
#m

In [ ]:
# Export map as html
#m.save(path_plots+'folium_map_CO2_'+exp_rand+'.html')

#### Delta experiment

In [ ]:
#m = folium.Map(location=[45.469262, 9.182007],
#               tiles='CartoDB Positron',
#               zoom_start=12,
#               attr='CartoDB')
               #png_enabled=True)

In [ ]:
#for index, row in df_plot_delta_co2.iterrows():
#    if row['edge_id'] in removed_edges:
#        folium.PolyLine([row['from'], row['to']],
#                         tooltip=row['road']+' - \u0394 CO2_m: {:.3e}'.format(row['delta']),
#                         color='black', weight=1, dashArray='5').add_to(m)
#    else:
#        folium.PolyLine([row['from'], row['to']],
#                         tooltip=row['road']+' - \u0394 CO2_m: {:.3e}'.format(row['delta']),
#                         color=row['color'], weight=1.5).add_to(m)

In [ ]:
#m

In [ ]:
# Export map as html
#m.save(path_plots+'folium_map_delta_CO2_'+exp_name+'.html')

#### Delta experiment rand

In [ ]:
#m = folium.Map(location=[45.469262, 9.182007],
#               tiles='CartoDB Positron',
#               zoom_start=12,
#               attr='CartoDB')
               #png_enabled=True)

In [ ]:
#for index, row in df_plot_delta_co2_rand.iterrows():
    #if row['edge_id'] in removed_edges_rand:
    #    folium.PolyLine([row['from'], row['to']],
    #                     tooltip=row['road']+' - \u0394 CO2_m: {:.3e}'.format(row['delta_rand']),
    #                     color='black', weight=1, dashArray='5').add_to(m)
    #else:
#    folium.PolyLine([row['from'], row['to']],
#                     tooltip=row['road']+' - \u0394 CO2_m: {:.3e}'.format(row['delta_rand']),
#                     color=row['color'], weight=1.5).add_to(m)

In [ ]:
#m

In [ ]:
# Export map as html
#m.save(path_plots+'folium_map_delta_CO2_'+exp_rand+'.html')